In [26]:
import numpy as np
up = np.array([1., 0.])
down = np.array([0., 1.])
state = np.kron(up, up)

CX = np.array([[1., 0., 0., 0.,], [0., 1., 0., 0.], [0., 0., 0., 1.], [0., 0., 1., 0.]])
I = np.eye(2)
X = np.array([[0., 1.], [1., 0.]])
H = 1/np.sqrt(2)*np.array([[1, 1], [1, -1]])

In [ ]:
number_of_qubits = 2 # find better way of reading this from the state or circuit
circ = QuantumCircuit(number_of_qubits)
circ.x([0])
circ.cx(0, 1)
circ.x([1])

print(circ.draw())

ops = []

def apply_circuit(state, circuit):
    ops = []
    for index, instruction in enumerate(circ):
        # index is the gate number in the total number of gates. 
        # instruction consists of something like (operation, qubit) with operation as (name='x', num_qubits=1, num_clbits=0, params=[])
        
        # Operator converts this operation into a matrix. 
        gate = Operator(instruction.operation) 
        if instruction.operation.name == 'cx':
            gate = CX # since they go from MSB to LSB, their cx is unusual. maybe there's a more elegant solution though.
        ops.append(instruction.operation.name)

        # get the index of the qubits applied
        qubit_indices = []
        num_qubits = instruction.operation._num_qubits
        n = 0

        # save index numbers in array
        while n < num_qubits:
            index = instruction.qubits[n]._index
            qubit_indices.append(index)
            # assume adjacent CNOTs for now; eventually add the exception for non-adjacent CNOTs
            n = n + 1
        final_matrix = kron_products(gate, qubit_indices, number_of_qubits)
        state = np.dot(final_matrix, state)
        print(state)
        print("mat = ", final_matrix)
    print(ops)
    return state

In [ ]:
import numpy as np
def kron_products(gate, indices, number_of_qubits):
    # this assumes we are given some fixed gate, with an array of indices this gate is applied to. 
    # For now, assume 2-qubit gates that are adjacent and increasing in order (that is, CNOT(2, 3), not CNOT(3, 2)). I assume that if this isn't the case, we will add SWAP gates to make this happen. 

    I = np.eye(2)

    start_index = 1
    if indices[0] == 0:
        op = gate
        if len(indices) == 2: # for two-qubit gates, start one later
            start_index = 2
    else:
        op = I

    marker = 0 # used to skip following loops for two-qubit gates 
    for n in range(start_index, number_of_qubits):
        if marker == 1:
            marker = 0
            continue
        if n in indices:
            op = np.kron(op, gate)
            if len(indices) == 2:
                marker = 1
        else:
            op = np.kron(op, I)
        
    return op

Tensorize things

In [45]:
import numpy as np
# take a state that is 2^N and divide into N sites

state = np.kron(np.kron(up, up), np.kron(up, up))
s = np.size(state)
if (s % 2 != 0):
    print(0)
N = int(s / 2)
state = np.reshape(state, (N, 2))
print(state)
print(np.tensordot(state, X, axes=([1], [0])))

[[1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 1.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [33]:
import numpy as np

# Example tensor A with shape (5, 5, 5, 5)
A = np.random.rand(5, 5, 5, 5)

# Example matrix M with shape (5, 5)
M = np.random.rand(5, 5)

# Perform the multiplication using tensordot
# A's third index (index 2) interacts with the first index of M (index 0)
result = np.tensordot(A, M, axes=([2], [0]))

np.shape(result)
# result will have the shape of the remaining dimensions of A and the second dimension of M
# resulting shape (5, 5, 5, 5)



(5, 5, 5, 5)